Conversion de PDF a imagenes 
    

In [6]:
import os
from pdf2image import convert_from_path

def convert_pdf_to_images(pdf_path, output_dir, dpi=400):
    """
    Convierte todas las páginas de un PDF a imágenes PNG.
    Args:
        pdf_path (str): Ruta al archivo PDF.
        output_dir (str): Carpeta donde se guardarán las imágenes.
        dpi (int): Resolución para la conversión. 300 suele ser un buen valor.
    Returns:
        List[str]: Lista de rutas a las imágenes generadas.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # pages será una lista de PIL Images
    pages = convert_from_path(pdf_path, dpi=dpi)

    image_paths = []
    for i, page in enumerate(pages):
        image_name = f"page_{i+1}.png"
        out_path = os.path.join(output_dir, image_name)
        # Guardar la imagen en PNG
        page.save(out_path, "PNG")
        image_paths.append(out_path)

    return image_paths


Carga el modelo de TATR de Hugging Face

In [7]:
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForObjectDetection

model_name = "microsoft/table-transformer-detection"
processor = AutoImageProcessor.from_pretrained(model_name, revision = "no_timm")  #no_timm se usa para evitar problemas 
model = AutoModelForObjectDetection.from_pretrained(model_name, revision = "no_timm")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

TableTransformerForObjectDetection(
  (model): TableTransformerModel(
    (backbone): TableTransformerConvModel(
      (conv_encoder): TableTransformerConvEncoder(
        (model): ResNetBackbone(
          (embedder): ResNetEmbeddings(
            (embedder): ResNetConvLayer(
              (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
              (normalization): TableTransformerFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          )
          (encoder): ResNetEncoder(
            (stages): ModuleList(
              (0): ResNetStage(
                (layers): Sequential(
                  (0): ResNetBasicLayer(
                    (shortcut): Identity()
                    (layer): Sequential(
                      (0): ResNetConvLayer(
                        (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(

Deteccion de tablas en cada imagen

In [8]:
import numpy as np

def detect_tables_in_image(image_path, confidence_threshold=0.3):
    """
    Detecta tablas en una imagen usando Table Transformer (detección).
    Args:
        image_path (str): Ruta a la imagen PNG/JPG.
        confidence_threshold (float): Umbral de confianza para filtrar predicciones.
    Returns:
        List[dict]: Lista de tablas detectadas, cada dict contiene 'score' y 'box' con coords (xmin, ymin, xmax, ymax).
    """
    image = Image.open(image_path).convert("RGB")
    # Preprocesar la imagen
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    # Postprocesar
    result = processor.post_process_object_detection(outputs, threshold=confidence_threshold)[0]

    detections = []
    for score, label, box in zip(result["scores"], result["labels"], result["boxes"]):
        if label == 1:  # 1 es la clase 'table' en este modelo
            # box es [xmin, ymin, xmax, ymax]
            det = {
                "score": float(score.cpu().numpy()),
                "box": [float(x) for x in box.cpu().numpy()]
            }
            detections.append(det)
    return detections


In [9]:
def crop_table_images(image_path, detections, output_dir):
    """
    Recorta y guarda las regiones de tabla detectadas en 'detections'.
    Args:
        image_path (str): Ruta a la imagen original.
        detections (List[dict]): Lista de tablas con su 'box'.
        output_dir (str): Carpeta donde guardar los recortes.
    Returns:
        List[str]: Rutas de las imágenes recortadas de cada tabla.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image = Image.open(image_path).convert("RGB")
    table_paths = []
    for i, det in enumerate(detections):
        box = det["box"]  # [xmin, ymin, xmax, ymax]
        xmin, ymin, xmax, ymax = box
        # Recortar
        cropped = image.crop((xmin, ymin, xmax, ymax))
        table_path = os.path.join(output_dir, f"table_{i+1}.png")
        cropped.save(table_path, "PNG")
        table_paths.append(table_path)
    return table_paths


In [10]:
def process_pdf_for_tables(pdf_path, base_output_dir="output_tables", dpi=400):
    """
    Convierte PDF a imágenes, detecta tablas en cada página y recorta.
    """
    # 1) Convertir a imágenes
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    pages_dir = os.path.join(base_output_dir, pdf_name, "pages")
    tables_dir = os.path.join(base_output_dir, pdf_name, "tables")

    page_images = convert_pdf_to_images(pdf_path, pages_dir, dpi=dpi)

    all_tables = []  # Para almacenar info de cada tabla
    for page_image in page_images:
        # 2) Detectar tablas
        detections = detect_tables_in_image(page_image, confidence_threshold=0.3)
        # 3) Recortar cada tabla detectada
        page_id = os.path.splitext(os.path.basename(page_image))[0]  # "page_1"
        page_tables_dir = os.path.join(tables_dir, page_id)
        table_paths = crop_table_images(page_image, detections, page_tables_dir)

        # Podrías almacenar metadatos (por ejemplo, la bounding box, etc.)
        for tpath, det in zip(table_paths, detections):
            all_tables.append({
                "pdf_name": pdf_name,
                "page": page_id,
                "score": det["score"],
                "box": det["box"],
                "table_image": tpath
            })

    return all_tables

# Ejemplo de ejecución
if __name__ == "__main__":
    pdf_file = "/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mazda" 
    result_tables = process_pdf_for_tables(pdf_file, base_output_dir="l200_output", dpi=400)
    print("Tablas detectadas:", result_tables)


PDFPageCountError: Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table


In [12]:
import os
import json
from pdf2image import convert_from_path
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModelForObjectDetection

# Configuración del modelo (igual que antes)
model_name = "microsoft/table-transformer-detection"
processor = AutoImageProcessor.from_pretrained(model_name, revision="no_timm")
model = AutoModelForObjectDetection.from_pretrained(model_name, revision="no_timm")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# Función para recolectar todas las imágenes procesadas
def collect_processed_images(base_output_dir):
    """Recolecta todas las rutas de imágenes procesadas para Label Studio"""
    image_data = []
    
    for root, dirs, files in os.walk(base_output_dir):
        if "pages" in root:  # Solo buscamos en directorios de páginas
            for file in files:
                if file.lower().endswith('.png'):
                    full_path = os.path.abspath(os.path.join(root, file))
                    image_data.append({
                        "image": full_path,
                        "pdf_name": os.path.basename(os.path.dirname(os.path.dirname(root))),
                        "original_pdf": os.path.join(
                            os.path.dirname(os.path.dirname(root)),
                            "original",
                            os.path.basename(root) + ".pdf"  # Asumiendo que guardas los PDFs originales
                        )
                    })
    return image_data

# Función principal modificada
def process_all_pdfs(root_dir, base_output_dir="output_tables", dpi=400):
    all_tables = []
    
    # Primero procesamos todos los PDFs
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                try:
                    print(f"🔍 Procesando: {pdf_path}")
                    pdf_name = os.path.splitext(file)[0]
                    output_pdf_dir = os.path.join(base_output_dir, pdf_name)
                    
                    # Convertir PDF a imágenes
                    pages_dir = os.path.join(output_pdf_dir, "pages")
                    convert_pdf_to_images(pdf_path, pages_dir, dpi)
                    
                    # Opcional: Guardar una copia del PDF original
                    original_pdf_dir = os.path.join(output_pdf_dir, "original")
                    os.makedirs(original_pdf_dir, exist_ok=True)
                    os.system(f'cp "{pdf_path}" "{original_pdf_dir}"')

                except Exception as e:
                    print(f"❌ Error procesando {pdf_path}: {str(e)}")
    
    # Luego generamos el JSON para Label Studio
    label_studio_data = []
    images = collect_processed_images(base_output_dir)
    
    for img in images:
        label_studio_data.append({
            "data": {
                "image": img["image"],
                "pdf_info": {
                    "name": img["pdf_name"],
                    "original_path": img["original_pdf"]
                }
            }
        })
    
    # Guardar el JSON
    json_path = os.path.join(base_output_dir, "label_studio_import.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(label_studio_data, f, indent=4, ensure_ascii=False)
    
    print(f"\n✅ JSON generado con {len(label_studio_data)} imágenes en: {json_path}")
    return all_tables

# Funciones auxiliares (las mismas que antes)
def convert_pdf_to_images(pdf_path, output_dir, dpi=400):
    os.makedirs(output_dir, exist_ok=True)
    pages = convert_from_path(pdf_path, dpi=dpi)
    return [page.save(os.path.join(output_dir, f"page_{i+1}.png"), "PNG") for i, page in enumerate(pages)]

def detect_tables_in_image(image_path, confidence_threshold=0.6):
    # (Implementación anterior)
    pass

def crop_table_images(image_path, detections, output_dir):
    # (Implementación anterior)
    pass

# Ejecución
if __name__ == "__main__":
    # Configuración
    input_root = "/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars"
    output_dir = "images_pdf"
    
    # Procesar todo y generar JSON
    process_all_pdfs(
        root_dir=input_root,
        base_output_dir=output_dir,
        dpi=400
    )

🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/17717-arrizo5pro_02012024.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/93159-tiggo8promax_16072024.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/67227-tiggo7pro_16072024.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/35617-new_tiggo8pro_02012024.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/29068-ficha_tiggo2pro_23012025.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/42299-tiggo4pro_16072024.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/80544-tiggo8pro_hybrid.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Cherry/24270-tiggo7prohybrid_26112024.pdf
🔍 Procesando: /home/pibezx/Documents/P

/home/pibezx/miniconda3/envs/chatbot_ml/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (103937500 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/pibezx/miniconda3/envs/chatbot_ml/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (125887500 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/pibezx/miniconda3/envs/chatbot_ml/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (150887500 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/pibezx/miniconda3/envs/chatbot_ml/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (106893750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/pibezx/miniconda3/envs/chatbot_ml/lib/python3.10/site-

🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Renault/oroch-ft.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Renault/ficha-tecnica_duster_compressed.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Renault/koleos-ft-1.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Chevrolet/202411-03-onix-turbo-my25.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Chevrolet/202411-12-colorado-brutal-my25.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Chevrolet/202411-13-silverado.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Chevrolet/202411-07-traverse.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Chevrolet/202411-06-captiva-xl.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles

cp: '/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/images_pdf/FT-Grand-i10-sedan/original/FT-Grand-i10-sedan.pdf' and 'images_pdf/FT-Grand-i10-sedan/original/FT-Grand-i10-sedan.pdf' are the same file


🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/images_pdf/FT-CRETA-FL-2024/original/FT-CRETA-FL-2024.pdf


...

🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/images_pdf/2024-09-13-ficha_x7plussetiembre2024/original/2024-09-13-ficha_x7plussetiembre2024.pdf


cp: '/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/images_pdf/2024-09-13-ficha_x7plussetiembre2024/original/2024-09-13-ficha_x7plussetiembre2024.pdf' and 'images_pdf/2024-09-13-ficha_x7plussetiembre2024/original/2024-09-13-ficha_x7plussetiembre2024.pdf' are the same file


🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/images_pdf/oroch-ft/original/oroch-ft.pdf


cp: '/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/images_pdf/oroch-ft/original/oroch-ft.pdf' and 'images_pdf/oroch-ft/original/oroch-ft.pdf' are the same file


🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT-Nueva-Frontier-PRO-4X.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT_X-TRAIL_E-POWER.pdf
❌ Error procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT_X-TRAIL_E-POWER.pdf: Image size (525970806 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT-Nueva-Frontier.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT-Urvan.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT-Nuevo-Sentra.pdf
🔍 Procesando: /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT-Nuevo-Nissan-Versa.pdf
❌ Error procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Nissan/FT-Nuevo-Nissan-